In [1]:
import apache_beam as beam


In [4]:
class MyTransform(beam.PTransform):
  
  def expand(self, input_coll):
    
    a = ( 
        input_coll
        | 'Group and sum1' >> beam.CombinePerKey(sum)
        | 'count filter accounts' >> beam.Filter(filter_on_count)
        | 'Regular accounts employee' >> beam.Map(format_output)
              
    )
    return a

def SplitRow(element):
    return element.split(',')
  
  
def filter_on_count(element):
  name, count = element
  if count > 30:
    return element
  
def format_output(element):
  name, count = element
  #return ', '.join((name.encode('ascii'),str(count),'Regular employee'))
  return ', '.join((name,str(count),'Regular employee'))

p = beam.Pipeline()

input_collection = ( 
                      p 
                      | "Read from text file" >> beam.io.ReadFromText('section-2/dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'composite accoubts' >> MyTransform()
                    #   | 'Write results for account' >> beam.io.WriteToText('data/Account')
                      | 'print 1' >> beam.Map(print)
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                | 'composite HR' >> MyTransform()
                # | 'Write results for hr' >> beam.io.WriteToText('data/HR')
                | 'print 2' >> beam.Map(print)
           ) 
p.run()

Accounts, Marco, 31, Regular employee
Accounts, Rebekah, 31, Regular employee
Accounts, Itoe, 31, Regular employee
Accounts, Edouard, 31, Regular employee
Accounts, Kyle, 62, Regular employee
Accounts, Kumiko, 31, Regular employee
Accounts, Gaston, 31, Regular employee
HR, Beryl, 62, Regular employee
HR, Olga, 31, Regular employee
HR, Leslie, 31, Regular employee
HR, Mindy, 31, Regular employee
HR, Vicky, 31, Regular employee
HR, Richard, 31, Regular employee
HR, Kirk, 31, Regular employee
HR, Kaori, 31, Regular employee
HR, Oscar, 31, Regular employee
